In [1]:
import boto3
import pandas as pd
from io import StringIO
from time import sleep

In [2]:
AWS_ACCESS_KEY='YOUR_ACCESS_KEY'
AWS_SECRET_KEY='YOUR_SECRET_KEY'
AWS_REGION='ap-south-1'
SCHEMA_NAME='covid_dataset_19'
S3_STAGING_DIR='s3://sanket-covid19-output/output/'
S3_BUCKET_NAME='sanket-covid19-output'
S3_OUTPUT_DIR='output'



In [3]:
athena_client= boto3.client(
    "athena",
     aws_access_key_id=AWS_ACCESS_KEY,
     aws_secret_access_key=AWS_SECRET_KEY,
     region_name=AWS_REGION
)

In [4]:
Dict={}

def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
        try:
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            print(err)
            if "not finish yet" in str(err):
                sleep(0.01)
            else:
                raise(err)
                
    temp_file_location:str="athena_query_results.csv"
    
    s3_client = boto3.client('s3',
                     aws_access_key_id=AWS_ACCESS_KEY,
                     aws_secret_access_key=AWS_SECRET_KEY,
                     region_name=AWS_REGION
                    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIR}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    
    return pd.read_csv(temp_file_location)

In [5]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)


In [6]:
response

{'QueryExecutionId': 'f3300a85-1c81-43a2-b7c2-ebc153843437',
 'ResponseMetadata': {'RequestId': '4b25b208-b074-4ab2-ae26-5ba543da0a11',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:48:30 GMT',
   'x-amzn-requestid': '4b25b208-b074-4ab2-ae26-5ba543da0a11',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [7]:
# Some data was missing from the zip file converet it to csv then extracted
enigma_jhud = download_and_load_query_results(athena_client,response)

In [8]:
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,"""""","""Anhui""","""China""","""2020-01-22T17:00:00""",NaN,NaN,NaN,NaN,NaN,"""""","""""""Anhui"""
1,NaN,"""""","""Beijing""","""China""","""2020-01-22T17:00:00""",NaN,NaN,NaN,NaN,NaN,"""""","""""""Beijing"""
2,NaN,"""""","""Chongqing""","""China""","""2020-01-22T17:00:00""",NaN,NaN,NaN,NaN,NaN,"""""","""""""Chongqing"""
3,NaN,"""""","""Fujian""","""China""","""2020-01-22T17:00:00""",NaN,NaN,NaN,NaN,NaN,"""""","""""""Fujian"""
4,NaN,"""""","""Gansu""","""China""","""2020-01-22T17:00:00""",NaN,NaN,NaN,NaN,NaN,"""""","""""""Gansu"""


In [9]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM countrycode",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [10]:
response

{'QueryExecutionId': 'c8145414-8d64-4c11-a575-d2a1f3a0fa4e',
 'ResponseMetadata': {'RequestId': '82601692-57f7-43c2-b0be-d76f06f106b9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:48:42 GMT',
   'x-amzn-requestid': '82601692-57f7-43c2-b0be-d76f06f106b9',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [11]:
countrycode = download_and_load_query_results(athena_client,response)
countrycode.head()

,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6


In [12]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM countypopulation",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [13]:
response

{'QueryExecutionId': '856d8f81-ad3c-4e10-b72f-dac555c4f2f5',
 'ResponseMetadata': {'RequestId': '4b4bff7b-f4b7-496a-b70e-46c8cb23f397',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:48:46 GMT',
   'x-amzn-requestid': '4b4bff7b-f4b7-496a-b70e-46c8cb23f397',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [14]:
countypopulation = download_and_load_query_results(athena_client,response)
countypopulation.head()

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


In [15]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [16]:
response

{'QueryExecutionId': '1ef2e4f6-270d-427a-aff6-04aa321c2e1d',
 'ResponseMetadata': {'RequestId': '98fa6c24-424f-4053-a3e2-6cd47786c32c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:48:52 GMT',
   'x-amzn-requestid': '98fa6c24-424f-4053-a3e2-6cd47786c32c',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [17]:
rearc_usa_hospital_beds = download_and_load_query_results(athena_client,response)
rearc_usa_hospital_beds.head()

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
0,2026,Meeker County Memorial Hospital,Critical Access Hospital,612 S Sibley Ave,NaN,Litchfield,MN,55355,Meeker,Minnesota,...,35.0,25.0,4,4,0.0,0.318795,0.0,10,45.119098,-94.528802
1,2027,Mille Lacs Health System,Critical Access Hospital,200 Elm St N,NaN,Onamia,MN,56359,Mille Lacs,Minnesota,...,28.0,18.0,5,5,0.0,0.370167,0.0,10,46.073181,-93.662491
2,2028,M Health Fairview Northland Medical Center,Short Term Acute Care Hospital,911 Northland Dr,NaN,Princeton,MN,55371,Sherburne,Minnesota,...,54.0,34.0,4,4,0.0,0.351974,2.0,20,45.558437,-93.590102
3,2029,CHI St Gabriel Health (FKA St Gabriels Hospital),Critical Access Hospital,815 2nd St Se,NaN,Little Falls,MN,56345,Morrison,Minnesota,...,49.0,25.0,4,4,0.0,0.262732,2.0,24,45.967300,-94.362700
4,2030,Murray County Medical Center,Critical Access Hospital,2042 Juniper Ave,NaN,Slayton,MN,56172,Murray,Minnesota,...,25.0,21.0,3,3,0.0,0.164123,2.0,4,43.992305,-95.759840


In [18]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM states_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)

In [19]:
response

{'QueryExecutionId': 'c41542cd-0782-4271-a0ce-d01401af6138',
 'ResponseMetadata': {'RequestId': '922ef4d5-a40b-4987-93e4-3a8b0f6e668a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:48:59 GMT',
   'x-amzn-requestid': '922ef4d5-a40b-4987-93e4-3a8b0f6e668a',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [20]:
states_daily = download_and_load_query_results(athena_client,response)
states_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0.0,0.0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0.0,0.0,0.0,0.0,0.0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1.0,0.0,997207b430824ea40b8eb8506c19a93e07bc972e,0.0,0.0,0.0,0.0,0.0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22.0,11.0,50921aeefba3e30d31623aa495b47fb2ecc72fae,0.0,0.0,0.0,0.0,0.0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0.0,0.0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0.0,0.0,0.0,0.0,0.0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5.0,44.0,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0.0,0.0,0.0,0.0,0.0,NaN


In [21]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [22]:
response

{'QueryExecutionId': '93b317f8-d369-487f-9150-607017b562ef',
 'ResponseMetadata': {'RequestId': '11672517-7638-4c0c-87aa-db165bdb54b6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:49:09 GMT',
   'x-amzn-requestid': '11672517-7638-4c0c-87aa-db165bdb54b6',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [23]:
us_county = download_and_load_query_results(athena_client,response)
us_county.head()

,date,county,state,fips,cases,deaths
0,2020-03-22,St. Charles,Missouri,29183.0,3.0,0.0
1,2020-03-22,St. Louis,Missouri,29189.0,55.0,1.0
2,2020-03-22,St. Louis city,Missouri,29510.0,14.0,0.0
3,2020-03-22,Unknown,Missouri,NaN,1.0,0.0
4,2020-03-22,Broadwater,Montana,30007.0,1.0,0.0


In [24]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [25]:
response

{'QueryExecutionId': 'd52121d7-fcf7-43e0-9c09-8a55115d4c1f',
 'ResponseMetadata': {'RequestId': '52e9b7f2-4fa0-4b89-9909-357af08aa4a8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:49:15 GMT',
   'x-amzn-requestid': '52e9b7f2-4fa0-4b89-9909-357af08aa4a8',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [26]:
us_daily = download_and_load_query_results(athena_client,response)
us_daily.head()

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [27]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [28]:
response

{'QueryExecutionId': '8733f5a0-54ed-410b-95e7-729e5f22e215',
 'ResponseMetadata': {'RequestId': '9c1a9be4-ffd3-4ea8-b58e-74994f517291',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:49:24 GMT',
   'x-amzn-requestid': '9c1a9be4-ffd3-4ea8-b58e-74994f517291',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [29]:
us_states = download_and_load_query_results(athena_client,response)
us_states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [30]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_total_latest",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)



In [31]:
response

{'QueryExecutionId': '511fcd91-f338-46c0-a63c-8b89875c5715',
 'ResponseMetadata': {'RequestId': 'efaed1a5-3030-4a35-9aad-e8d6e41bbe85',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:49:30 GMT',
   'x-amzn-requestid': 'efaed1a5-3030-4a35-9aad-e8d6e41bbe85',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [32]:
us_total_latest = download_and_load_query_results(athena_client,response)
us_total_latest.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [33]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM state_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation":S3_STAGING_DIR,
        "EncryptionConfiguration":{"EncryptionOption":"SSE_S3"},
    },
)


In [34]:
response

{'QueryExecutionId': '3e18ee53-2eb8-48b5-ba58-7f490c4aa51f',
 'ResponseMetadata': {'RequestId': 'c2c3c40c-e82d-4546-89aa-2ce3b0ead3be',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 14 May 2022 15:49:40 GMT',
   'x-amzn-requestid': 'c2c3c40c-e82d-4546-89aa-2ce3b0ead3be',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [35]:
state_abv = download_and_load_query_results(athena_client,response)
state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [36]:
# Change the header for state_abv as the 1 st row has STATE ABBREVIATEION AS VALUE

newheader = state_abv.iloc[0] # Grab the 1st row
state_abv =state_abv[1:]      # Set the dataframe from 2nd row
state_abv.columns= newheader  # Set the columns header as the first row


In [37]:
state_abv.head()  # See above table for reference

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


In [38]:
factCovid_1 = enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
factCovid_2 = states_daily[['fips','date','positive', 'negative','hospitalizedcurrently','hospitalized',
       'hospitalizeddischarged']]

factCovid = pd.merge(factCovid_1,factCovid_2,on='fips',how='inner') #Here we join that two dataframes..
factCovid.head()

,fips,province_state,country_region,confirmed,deaths,recovered,active,date,positive,negative,hospitalizedcurrently,hospitalized,hospitalizeddischarged
0,NaN,"""Anhui""","""China""",NaN,NaN,NaN,"""""",20210119,289939,NaN,1066.0,NaN,NaN
1,NaN,"""Beijing""","""China""",NaN,NaN,NaN,"""""",20210119,289939,NaN,1066.0,NaN,NaN
2,NaN,"""Chongqing""","""China""",NaN,NaN,NaN,"""""",20210119,289939,NaN,1066.0,NaN,NaN
3,NaN,"""Fujian""","""China""",NaN,NaN,NaN,"""""",20210119,289939,NaN,1066.0,NaN,NaN
4,NaN,"""Gansu""","""China""",NaN,NaN,NaN,"""""",20210119,289939,NaN,1066.0,NaN,NaN


In [39]:
dimRegion_1 = enigma_jhud[['fips', 'province_state', 'country_region','latitude', 'longitude']]
dimRegion_2 = us_county[['fips','state','county']]
dimRegion =pd.merge(dimRegion_1,dimRegion_2,on='fips',how='inner')
dimRegion.shape

(90458424, 7)

In [40]:
dimHospital = rearc_usa_hospital_beds[['fips', 'state_name', 'latitude', 'longtitude', 'hospital_name', 'hospital_type', 'hq_address','hq_city', 'hq_state',]]
dimHospital.shape

(6637, 9)

In [43]:
# Customized Date values like YYYY-MM-DD 
dimDate=states_daily[['fips','date']]
dimDate.head()

,fips,date
0,2.0,20210307
1,1.0,20210307
2,5.0,20210307
3,60.0,20210307
4,4.0,20210307


In [46]:
dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')

dimDate['year'] =dimDate['date'].dt.year

dimDate['month'] =dimDate['date'].dt.month

dimDate['day_of_week'] =dimDate['date'].dt.dayofweek

dimDate.head()
# Used Pandas to_datetime for readable format
# Ignore warning

C:\Users\Sanket Dhabale\AppData\Local\Temp\ipykernel_8400\61843209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['date']=pd.to_datetime(dimDate['date'],format='%Y%m%d')
C:\Users\Sanket Dhabale\AppData\Local\Temp\ipykernel_8400\61843209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimDate['year'] =dimDate['date'].dt.year
C:\Users\Sanket Dhabale\AppData\Local\Temp\ipykernel_8400\61843209.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

,fips,date,year,month,day_of_week
0,2.0,2021-03-07,2021,3,6
1,1.0,2021-03-07,2021,3,6
2,5.0,2021-03-07,2021,3,6
3,60.0,2021-03-07,2021,3,6
4,4.0,2021-03-07,2021,3,6


In [48]:
bucket = 'sanket-covid19-output'
csv_buffer=StringIO()
factCovid.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                     region_name="ap-south-1",
                     aws_access_key_id=AWS_ACCESS_KEY,
                     aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket,'output/factCovid.csv').put(Body=csv_buffer.getvalue())


{'ResponseMetadata': {'RequestId': 'PKC2KK2SZ6RHH3RR',
  'HostId': 'OxipSapv4gBLWClq7AINrcW+HKdbxA3cJxsQ2MfVqFvcBq2dVaJYzY+W1WIcC52vlpFgaBa+w8k=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'OxipSapv4gBLWClq7AINrcW+HKdbxA3cJxsQ2MfVqFvcBq2dVaJYzY+W1WIcC52vlpFgaBa+w8k=',
   'x-amz-request-id': 'PKC2KK2SZ6RHH3RR',
   'date': 'Sat, 14 May 2022 17:27:11 GMT',
   'etag': '"5e485fe4622912305327b462e0729608"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"5e485fe4622912305327b462e0729608"'}

In [49]:
csv_buffer=StringIO()
dimRegion.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                     region_name="ap-south-1",
                     aws_access_key_id=AWS_ACCESS_KEY,
                     aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket,'output/dimRegion.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'KCKHW0GV4TJSBYMW',
  'HostId': 'tLTl1NrSKJ73B99xt13rjXYLYIChVBRy2Z+HZqIoyF2dCXBNgi0YCIkIbK2pOOPpixFz8rhyzIg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'tLTl1NrSKJ73B99xt13rjXYLYIChVBRy2Z+HZqIoyF2dCXBNgi0YCIkIbK2pOOPpixFz8rhyzIg=',
   'x-amz-request-id': 'KCKHW0GV4TJSBYMW',
   'date': 'Sat, 14 May 2022 18:01:15 GMT',
   'etag': '"d5351024b741f464f85a9a01a31ff20f"',
   'server': 'AmazonS3',
   'content-length': '0',
   'connection': 'close'},
  'RetryAttempts': 0},
 'ETag': '"d5351024b741f464f85a9a01a31ff20f"'}

In [50]:
csv_buffer=StringIO()
dimDate.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                     region_name="ap-south-1",
                     aws_access_key_id=AWS_ACCESS_KEY,
                     aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket,'output/dimDate.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'XZ8G9W7H0FG25EV2',
  'HostId': 'F9w0jXTZImO/CHQwp6z3IaM3oTkKZmP1l/2gKudvJo2+csrgMvzNZqx6lJNGmN6Ebf6f1eCb1BI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'F9w0jXTZImO/CHQwp6z3IaM3oTkKZmP1l/2gKudvJo2+csrgMvzNZqx6lJNGmN6Ebf6f1eCb1BI=',
   'x-amz-request-id': 'XZ8G9W7H0FG25EV2',
   'date': 'Sat, 14 May 2022 18:48:18 GMT',
   'etag': '"19eb0b77e7f7441c686829bc3fd1a906"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"19eb0b77e7f7441c686829bc3fd1a906"'}

In [51]:
csv_buffer=StringIO()
dimHospital.to_csv(csv_buffer)
s3_resource = boto3.resource('s3',
                     region_name="ap-south-1",
                     aws_access_key_id=AWS_ACCESS_KEY,
                     aws_secret_access_key=AWS_SECRET_KEY)
s3_resource.Object(bucket,'output/dimHospital.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '6HSJVEQHJ0ZPQK36',
  'HostId': 'cGo1Ms4VgWF3h0Y5vUnfqKUr0rDQqSPqjiD5ZDZKBnc0hKASqNG/8gVege9P11fMrrY377RTI0o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cGo1Ms4VgWF3h0Y5vUnfqKUr0rDQqSPqjiD5ZDZKBnc0hKASqNG/8gVege9P11fMrrY377RTI0o=',
   'x-amz-request-id': '6HSJVEQHJ0ZPQK36',
   'date': 'Sat, 14 May 2022 18:48:19 GMT',
   'etag': '"47a6fd09658475a3e2958a1461dc71af"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"47a6fd09658475a3e2958a1461dc71af"'}

In [52]:
dimDateSQL = pd.io.sql.get_schema(dimDate.reset_index(),'dimDate')
print(''.join(dimDateSQL))

CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)


In [53]:
dimHospitalSQL = pd.io.sql.get_schema(dimHospital.reset_index(),'dimHospitalSQL')
print(''.join(dimHospitalSQL))

CREATE TABLE "dimHospitalSQL" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_address" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)


In [54]:
dimRegionSQL = pd.io.sql.get_schema(dimRegion.reset_index(),'dimRegion')
print(''.join(dimRegionSQL))

CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "state" TEXT,
  "county" TEXT
)


In [55]:
factCovidSQL = pd.io.sql.get_schema(factCovid.reset_index(),'factCovid')
print(''.join(factCovidSQL))

CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" TEXT,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)


In [56]:
pip install redshift_connector

You should consider upgrading via the 'C:\Users\Sanket Dhabale\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



     ------------------------------------ 109.9/109.9 KB 426.0 kB/s eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 3.1 MB/s eta 0:00:00
     ------------------------------------ 105.0/105.0 KB 551.2 kB/s eta 0:00:00


In [57]:
import redshift_connector

In [67]:
conn = redshift_connector.connect(
    host='my-first-redshift.clytzlhnirzm.ap-south-1.redshift.amazonaws.com',
    database='myfisrtdb',
    user='awsuser',
    password='Sanket15'
 )

In [68]:
conn.autocommit=True

In [69]:
cursor= redshift_connector.Cursor=conn.cursor()

In [70]:
cursor.execute("""
CREATE TABLE "dimDate" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
""")

cursor.execute("""
CREATE TABLE "dimHospitalSQL" (
"index" INTEGER,
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_address" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")


cursor.execute("""
CREATE TABLE "dimRegion" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "state" TEXT,
  "county" TEXT
)
""")


cursor.execute("""
CREATE TABLE "factCovid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" TEXT,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
""")

In [74]:
cursor.execute("""
copy dimDate from 's3://sanket-covid19-output/output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::500611132176:role/redshift-s3Access'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")


In [76]:

cursor.execute("""
copy dimHospitalSQL from 's3://sanket-covid19-output/output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::500611132176:role/redshift-s3Access'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")


In [78]:

cursor.execute("""
copy factCovid from 's3://sanket-covid19-output/output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::500611132176:role/redshift-s3Access'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")

In [81]:
cursor.execute("""
copy dimRegion from 's3://sanket-covid19-output/output/dimRegion.csv'
credentials 'aws_iam_role=arn:aws:iam::500611132176:role/redshift-s3Access'
delimiter ','
region 'ap-south-1'
IGNOREHEADER 1
""")
